In [3]:
import cv2
import numpy as np
from PIL import Image

In [4]:
def classify_flag(flag_image):
    image = Image.fromarray(flag_image)
    image = image.convert("RGB")

    # Resize the image for uniformity
    image = image.resize((100, 100))

    # Split the image into top and bottom halves
    top_half = image.crop((0, 0, 100, 50))
    bottom_half = image.crop((0, 50, 100, 100))

    # Calculate the average RGB of the top half
    def average_rgb(img):
        pixels = np.array(img)
        r, g, b = pixels[:, :, 0], pixels[:, :, 1], pixels[:, :, 2]
        return (r.mean(), g.mean(), b.mean())

    top_avg = average_rgb(top_half)
    bottom_avg = average_rgb(bottom_half)

    # Classify the flag based on the color of the top and bottom parts
    if top_avg[0] > 150 and top_avg[1] < 100 and top_avg[2] < 100:
        print("Indonesia Flag")
    else:
        print("Poland Flag")

In [5]:
def detect_flag(image_path):
    image = cv2.imread(image_path)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    edges = cv2.Canny(gray, 50, 150)
    contours, _ = cv2.findContours(edges, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    for contour in contours:
        epsilon = 0.01 * cv2.arcLength(contour, True)
        approx = cv2.approxPolyDP(contour, epsilon, True)
        if len(approx) == 4:
            x, y, w, h = cv2.boundingRect(approx)
            aspect_ratio = float(w) / h
            if 1.5 < aspect_ratio < 3:
                cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)
                flag_region = image[y:y + h, x:x + w]
                cv2.imshow('Detected Flag', image)
                classify_flag(flag_region)
                cv2.waitKey(0)
                cv2.destroyAllWindows()
                return
    print("No flag detected in the image.")
    cv2.destroyAllWindows()

In [ ]:
image_path = "image_path"  
detect_flag(image_path)